# [모듈 1.1] CodeCommit을 하면 EventBridge를 통해 CodePipeline 수행

이 노트북은 CodeCommit 레파지토리에 'SageMaker Pipeline config 파일(sm_pipeline_train_config.json)'을 수정 후 push 하면, EventBridge를 통해 CodePipeline을 실행하는 노트북 입니다.

## 참고 자료
- Amazon EventBridge 유저 가이드
   (https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-what-is.html)
- Boto3 Docs : EventBridge
   (https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/events.html)



# 1. 환경 확인

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import sagemaker
import boto3
sys.path.append('./src')

%store -r s3_input_data_uri
print("s3_input_data_uri: n", s3_input_data_uri)


# 2. 리파지토리 로컬에 클로닝

### repository_url :
```
repository_url =  "https://git-codecommit.<REGION>.amazonaws.com/v1/repos/ncf-training-repo"
```

In [ ]:
# 이전 4_sm-train-codepipeline 에서 만든 훈련 용 repository name 로딩
%store -r training_repository_name

# Codecommit Repo Name (ncf-training-repo)
repository_name = training_repository_name
#repository_name = "ncf-training-repo"

print(repository_name)

In [ ]:
repository_url = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('cloneUrlHttp')
print(repository_url)

# repository_url = "<Repository URL>"

In [ ]:
%%sh -s {repository_url} {repository_name}

repository_url=$1
repository_name=$2

rm -rf $repository_name

git clone $repository_url 

# 3. 파라미터 세팅

In [ ]:

# code pipeline name, arn, role_arn
codepipeline_name1 = "ncf-code-pipeline-training-sm-pipeline"
codepipeline_arn = boto3.client('codepipeline').get_pipeline(name = codepipeline_name1).get('metadata').get('pipelineArn')
eventbrideexcrole_arn = boto3.client('iam').get_role(RoleName = 'MLOps-EventBridgeRole').get('Role').get('Arn')

# EventBridge Rule Name
eventbridge_rule_name1 = "codepipelinerule1"

# Training data path (Prefix)
data_prefix = "NCFModel/data2"
training_files = 'codecommit1'

# 4. EventBridge Rule 정의

## 4.1 EventPattern 정의

In [ ]:
# Codecommit Arn
repository_arn = boto3.client('codecommit').get_repository(repositoryName = repository_name).get('repositoryMetadata').get('Arn')
print(repository_arn)

#### 위 CodeCommit arn 출력값을 복사해서 아래 resources 부분에 붙여넣으세요.

In [ ]:
eventclient = boto3.client('events')

eventresponse = eventclient.put_rule(
    Name = eventbridge_rule_name1,
    EventPattern = """{
        "source": ["aws.codecommit"],
        "detail-type": ["CodeCommit Repository State Change"],
        "resources": ["arn:aws:codecommit:us-east-1:376278017302:ncf-training-repo"],
        "detail": {
            "referenceType": ["branch"],
            "referenceName": ["master"]
        }
    }"""
)

print(eventresponse)

## 4.2 Event Target 설정

In [ ]:
eventresponse = eventclient.put_targets(
    Rule = eventbridge_rule_name1,
    Targets = [
        {
            'Arn' : codepipeline_arn,
            'Id' : codepipeline_name1,
            'RoleArn' : eventbrideexcrole_arn
        }
    ]
)

print(eventresponse)

# 5. 새로운 SageMaker Pipeline (Training) 구성 파일 준비

## 5.1. 변수 설정 및 dict 로 저장

In [ ]:
project_prefix = "SageMaker-Train-NCF"
bucket = sagemaker.Session().default_bucket()

train_sm_pipeline_name = "ncf-training-code-pipeline-sm-pipeline"
ModelApprovalStatus="PendingManualApproval"
inference_image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-gpu-py3"
training_instance_type = "ml.p3.2xlarge"
training_instance_count = 1

sm_pipeline_train_config_json_path = f'src/sm_pipeline_train_config.json'

In [ ]:
sm_train_pipeline_dict = {
    "project_prefix" : project_prefix,            
    "s3_input_data_uri" : s3_input_data_uri,
    "sm_pipeline_name" : train_sm_pipeline_name,
    "training_instance_type" :  training_instance_type,    
    "training_instance_count" :  training_instance_count,        
    "ModelApprovalStatus" :  ModelApprovalStatus,    
    "inference_image_uri" :  inference_image_uri,
    "bucket" : bucket,            
}

## 5.2. 설정 딕션너리 Json 파일로 저장 후 테스트를 위해 로딩

In [ ]:
from common_utils import save_json, load_json

save_json(sm_pipeline_train_config_json_path, sm_train_pipeline_dict)
sm_pipeline_train_dict = load_json(sm_pipeline_train_config_json_path)
print (json.dumps(sm_pipeline_train_dict, indent=2))

## 5.3 Config 파일 복사

### 5.3.1 codecommit1 폴더 생성

In [ ]:
newdir = training_files+"/pipelines/ncf/src/"
! rm -rf $newdir
! mkdir -p $newdir

### 5.3.2 sm_pipeline_train_config_json 파일 Copy

In [ ]:
source_path = sm_pipeline_train_config_json_path
target_path = newdir

! cp {source_path} {target_path}

# 6. SageMaker Pipeline config 파일을 로컬 레파지토리 디렉토리에 복사

In [ ]:
%%sh -s {repository_name} {training_files}
repository_name=$1
training_files=$2

cp -r $training_files/* $repository_name
ls $repository_name

# 7. 리파지토리에 푸시 (이벤트 트리거)

In [ ]:
%%sh -s {repository_name} 
repository_name=$1
cd $repository_name
date >> pushtime.log
echo $PWD
git add .
git commit -m "Add new files"
git push origin master


# 8. 변수 저장

In [ ]:
%store eventbridge_rule_name1
%store codepipeline_name1